In [ ]:
!pip install --upgrade openai


In [ ]:
!pip install pandas

In [ ]:
# from transformers import pipeline

# import pandas as pd
# import re

# # nlp = pipeline("sentiment-analysis", model="bardsai/finance-sentiment-es-base")
# nlp = pipeline(
#     "sentiment-analysis",
#     model="StephanAkkerman/FinTwitBERT-sentiment",
# )

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/713k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [ ]:
# from transformers import pipeline

# nlp = pipeline("sentiment-analysis", model="bardsai/twitter-sentiment-pl-base")
# nlp("Nigdy przegrana nie sprawiła mi takiej radości. Szczęście i Opatrzność mają znaczenie Gratuluje @pzpn_pl")


[{'label': 'positive', 'score': 0.9997233748435974}]

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
# model = AutoModelForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# from transformers import BertTokenizer, BertForSequenceClassification
# from transformers import pipeline

# def sentiment_analysis(text):
#     # tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
#     # model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
#     # model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
#     # tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

#     nlp = pipeline("sentiment-analysis")

#     return nlp(text)




In [ ]:
# print(sentiment_analysis("I love you"))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998656511306763}]


In [ ]:
from openai import OpenAI
import openai
# from openai.error import RateLimitError, APIError, InvalidRequestError
import json


def sentiment_analysis(usr_input):
    prompt = """
    I will provide conversational sentences from Facebook posts. You will perform sentiment analysis, classifying each post into one of three categories: **Positive**, **Neutral**, or **Negative**, and provide a score from 0.0 to 1.0 based on the strength of the sentiment. Return the sentiment category with the highest confidence score.

    ### Definitions:
    1. **Positive Sentiment**: Posts expressing happiness, excitement, gratitude, hope, relief, or any other emotion that conveys a positive experience or outlook. Examples include praise, expressions of love, enjoyment, or support.
    2. **Neutral Sentiment**: Posts that are informational, factual, or do not convey a strong emotional tone. These include general observations, updates, questions, or comments that neither imply a positive nor negative sentiment.
    3. **Negative Sentiment**: Posts expressing sadness, frustration, anger, jealousy, anxiety, disappointment, or any emotion that conveys a negative experience or outlook. This includes complaints, criticism, or expressions of distress.

    ### Examples:
    Q: "This phone is awesome"
    A: {"label": "Positive", "score": 1.0}

    Q: "I had a rough day today"
    A: {"label": "Negative", "score": 0.9}

    Q: "Snacking on watermelon watching house hunters lol"
    A: {"label": "Neutral", "score": 1.0}

    Q: "I miss you nuu nuu!!!"
    A: {"label": "Negative", "score": 0.8}

    Q: "Yumm — at City Cafe."
    A: {"label": "Positive", "score": 0.9}

    Q: "Dislike. Jealous."
    A: {"label": "Negative", "score": 1.0}

    Q: "People!! What's going on tonight?!"
    A: {"label": "Neutral", "score": 0.8}

    Return only the JSON object with the sentiment label and score for each input.


    """
    api_key = ''
    client = OpenAI(api_key=api_key)
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": usr_input},
    ]
)

    return json.loads(completion.choices[0].message.content)


In [ ]:
from json.decoder import JSONDecodeError
import pandas as pd
import re
# Define the sentiment analysis function (stub function)


# Load the Excel file
file_path = "/content/drive/MyDrive/collab_code/Final Database/1st_8k.xlsx"  # Change this path if needed
df = pd.read_excel(file_path)

# Function to preprocess text
def preprocess_text(text):
    # Remove links (delete the row if a link is found)
    if "http" in text or "www." in text:
        return None

    # Remove the pattern like [56] (number inside square brackets)
    text = re.sub(r"\[\d+\]", "", text).strip()

    return text

# Iterate through each row and process the 'Message' column
sentiments = []
scores = []
for index, row in df.iterrows():
    message = row["Message"]

    # Preprocess the message text
    processed_message = preprocess_text(message)

    # Skip the row if it contains a link (processed_message is None)
    if processed_message is None:
        df.drop(index, inplace=True)
        continue

    # Get the sentiment from the LLM function
    try:
      sentiment_score = sentiment_analysis(processed_message)
      sentiment = sentiment_score['label']
      score = sentiment_score['score']
    except JSONDecodeError:
      sentiment = "Undefined"
      score = 1.0
      continue
    except Exception as e:
      error_message = str(e)
      if "quota" in error_message.lower() or "rate limit" in error_message.lower():
          print("API quota exceeded or rate limit hit. Retrying after a short wait...")
          break
      else:
          print(f"An unexpected error occurred: {error_message}")
          break

    # Append the sentiment to the list
    sentiments.append(sentiment)
    scores.append(score)

    # Update the sentiment in the DataFrame
    df.at[index, "Sentiment"] = sentiment
    df.at[index, "Score"] = score

# Save the updated DataFrame back to the Excel file
output_path = "/content/drive/MyDrive/collab_code/Final Database/processed_sentiments1.xlsx"
df.to_excel(output_path, index=False)

print(f"Processing complete. The updated file is saved at: {output_path}")


API quota exceeded or rate limit hit. Retrying after a short wait...
Processing complete. The updated file is saved at: /content/drive/MyDrive/collab_code/Final Database/processed_sentiments1.xlsx
